In [70]:
import pandas as pd
import numpy as np
import random
import networkx as nx
from tqdm import tqdm
import matplotlib.pyplot as plt
from seaborn import heatmap, boxplot

from imblearn.under_sampling import RandomUnderSampler

from itertools import combinations

from node2vec import Node2Vec

from FeatureGeneration import add_all_feature, add_feature, generate_feature

from stellargraph.data import EdgeSplitter

## Train and Test split of the Graph

In [76]:
# load the train data
file = open('train.txt', 'r')
lines = file.readlines()

# find unique author pairs and count the number of papers coauthored
aPairs = {}
for line in lines:
    coAuthors = list(map(int, line.split()))
    for pair in combinations(coAuthors,2):
        pair = tuple(sorted(pair))
        if pair in aPairs:
            aPairs[pair] += 1
        else:
            aPairs[pair] = 1

In [77]:
# nca: the number of pairs coauthored
# exist: whether the pair of authors has coauthored
edges = pd.DataFrame(list(aPairs.items()), columns=['Pair', 'NCA'])
edges[['Source', 'Sink']] = pd.DataFrame(edges['Pair'].tolist(), index=edges.index)
edges = edges[["Source", "Sink", "NCA"]]

In [78]:
# df of existent edges
edges.head()

,Source,Sink,NCA
0,0,356,14
1,0,1236,14
2,356,1236,14
3,0,1655,9
4,0,1797,4


In [80]:
# create graph
G_train = nx.from_pandas_edgelist(edges, "Source", "Sink", create_using=nx.Graph())

# list of all nodes
nodeL = sorted(G_train.nodes())

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 3767
Number of edges: 16036
Average degree:   8.5139


In [81]:
# Test Graph

# Define an edge splitter on the original graph G
edge_splitter_test = EdgeSplitter(G)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from graph,
# and obtain the reduced graph graph_test with the sampled links removed:
G_test, edges_test, labels_test = edge_splitter_test.train_test_split(p=0.1, method="global",
                                                                      keep_connected=True, seed=42)
print(nx.info(G_test))

** Sampled 1603 positive and 1603 negative edges. **
Name: 
Type: Graph
Number of nodes: 3767
Number of edges: 14433
Average degree:   7.6629


In [82]:
# Train Graph
# Do the same process to compute a training subset from within the test graph
edge_splitter_train = EdgeSplitter(G_test, G)
G_train, edges_train, labels_train = edge_splitter_train.train_test_split(p=0.1, method="global",
                                                                 keep_connected=True, seed=42)
print(nx.info(G_train))

** Sampled 1443 positive and 1443 negative edges. **
Name: 
Type: Graph
Number of nodes: 3767
Number of edges: 12990
Average degree:   6.8967


In [83]:
train = pd.DataFrame(edges_train, columns = ['Source', 'Sink'])
train['Pair'] = list(zip(train.Source, train.Sink))
train['Exist'] = labels_train
train.head()

,Source,Sink,Pair,Exist
0,3460,2537,"(3460, 2537)",1
1,1173,3034,"(1173, 3034)",1
2,198,1398,"(198, 1398)",1
3,1724,1548,"(1724, 1548)",1
4,1842,2451,"(1842, 2451)",1


In [84]:
test = pd.DataFrame(edges_test, columns = ['Source', 'Sink'])
test['Pair'] = list(zip(test.Source, test.Sink))
test['Exist'] = labels_test
test.head()

,Source,Sink,Pair,Exist
0,1583,1090,"(1583, 1090)",1
1,141,3675,"(141, 3675)",1
2,1090,3599,"(1090, 3599)",1
3,2754,2434,"(2754, 2434)",1
4,3333,1354,"(3333, 1354)",1


In [85]:
kaggle = pd.read_csv("test-public.csv", index_col='Id')
kaggle["Pair"] = list(zip(kaggle.Source, kaggle.Sink))
kaggle.head()

,Source,Sink,Pair
Id,,,
1,0,2917,"(0, 2917)"
2,0,2956,"(0, 2956)"
3,1,4038,"(1, 4038)"
4,2,1848,"(2, 1848)"
5,3,513,"(3, 513)"


In [86]:
# build adjacency matrix
adj_G_train = nx.to_numpy_matrix(G_train, nodelist = nodeL)
adj_G_test = nx.to_numpy_matrix(G_test, nodelist = nodeL)
adj_G = nx.to_numpy_matrix(G, nodelist = nodeL)

## Feature Generation

### Similarity Based Features

In [89]:
train = add_all_feature(G_train, adj_G_train, nodeL, train)
train.head()

100%|██████████| 2886/2886 [00:00<00:00, 115652.43it/s]


,Source,Sink,Pair,Exist,CN,AA,RA,JC,PA,KI,PR
0,3460,2537,"(3460, 2537)",1,1,0.234594,0.014085,0.047619,96,0.000000,0.000353
1,1173,3034,"(1173, 3034)",1,37,9.603520,0.820722,0.308333,6150,0.004631,0.001767
2,198,1398,"(198, 1398)",1,2,0.883168,0.209524,0.050000,152,0.007974,0.000654
3,1724,1548,"(1724, 1548)",1,0,0.000000,0.000000,0.000000,102,0.000000,0.000523
4,1842,2451,"(1842, 2451)",1,3,1.964017,0.650000,0.500000,18,0.008433,0.000275


In [90]:
test = add_all_feature(G_test, adj_G_test, nodeL, test)
test.head()

100%|██████████| 3206/3206 [00:00<00:00, 114692.89it/s]


,Source,Sink,Pair,Exist,CN,AA,RA,JC,PA,KI,PR
0,1583,1090,"(1583, 1090)",1,11,3.205829,0.377772,0.392857,378,0.000000,0.000515
1,141,3675,"(141, 3675)",1,12,3.838447,0.545797,0.545455,280,0.089506,0.000352
2,1090,3599,"(1090, 3599)",1,2,0.649930,0.093306,0.111111,36,0.012238,0.000282
3,2754,2434,"(2754, 2434)",1,2,0.809981,0.185535,0.111111,51,0.000000,0.000281
4,3333,1354,"(3333, 1354)",1,2,0.982537,0.271739,0.039216,196,0.000000,0.000716


In [100]:
kaggle = add_all_feature(G, adj_G, nodeL, kaggle)
kaggle.head()

100%|██████████| 2000/2000 [00:00<00:00, 105358.05it/s]


,Source,Sink,Pair,CN,AA,RA,JC,PA,KI,PR
Id,,,,,,,,,,
1,0,2917,"(0, 2917)",0,0.00000,0.0,0.00,56,-0.000552,0.000202
2,0,2956,"(0, 2956)",0,0.00000,0.0,0.00,24,-0.000088,0.000173
3,1,4038,"(1, 4038)",0,0.00000,0.0,0.00,496,0.009896,0.000591
4,2,1848,"(2, 1848)",2,1.24267,0.4,0.08,72,0.057307,0.000332
5,3,513,"(3, 513)",0,0.00000,0.0,0.00,391,-0.138788,0.000454


### Node2Vec Features

#### Train

In [ ]:
ds = [20]
for d in ds:
    node2vec = Node2Vec(G_train, dimensions=d, walk_length=16, num_walks=100)
    n2v = node2vec.fit(window=7, min_count=1)
    n2vD = {}
    for u, v in train.Pair:
        # average is used as binary operator for learning edge features
        n2vD[(u, v)] = (n2v.wv.get_vector(str(u)) + n2v.wv.get_vector(str(v)))/2

    n2vDF = pd.DataFrame.from_dict(n2vD, orient='index', columns=["n2v_" + str(i+1) for i in range(d)])
    df = train.join(n2vDF, on='Pair')

    # export to csv
    df.to_csv("train_uw_{}.csv".format(d), index=False)

#### Test

In [87]:
ds = [20]
for d in ds:
    node2vec = Node2Vec(G_test, dimensions=d, walk_length=16, num_walks=100)
    n2v = node2vec.fit(window=7, min_count=1)
    n2vD = {}
    for u, v in test.Pair:
        # average is used as binary operator for learning edge features
        n2vD[(u, v)] = (n2v.wv.get_vector(str(u)) + n2v.wv.get_vector(str(v)))/2

    n2vDF = pd.DataFrame.from_dict(n2vD, orient='index', columns=["n2v_" + str(i+1) for i in range(d)])
    df = test.join(n2vDF, on='Pair')

    # export to csv
    df.to_csv("test_uw_{}.csv".format(d), index=False)

Computing transition probabilities:   0%|          | 0/3767 [00:00<?, ?it/s]


Generating walks (CPU: 1):  28%|██▊       | 28/100 [12:38<32:30, 27.09s/it][A

Generating walks (CPU: 1): 100%|██████████| 100/100 [04:23<00:00,  2.64s/it]


#### Kaggle

In [64]:
ds = [20]
for d in ds:
    node2vec = Node2Vec(G, dimensions=d, walk_length=16, num_walks=100)
    n2v = node2vec.fit(window=7, min_count=1)
    n2vD = {}
    for u, v in kaggle.Pair:
        if G.has_node(u):
            vec_u = n2v.wv.get_vector(str(u))
        else:
            vec_u = np.array([0]*d)

        if G.has_node(v):
            vec_v = n2v.wv.get_vector(str(v))
        else:
            vec_v = np.array([0]*d)
        
        # average is used as binary operator for learning edge features
        n2vD[(u, v)] = (vec_u + vec_v)/2

    n2vDF = pd.DataFrame.from_dict(n2vD, orient='index', columns=["n2v_" + str(i+1) for i in range(d)])
    df = kaggle.join(n2vDF, on='Pair')

    # export to csv
    df.to_csv("kaggle_uw_{}.csv".format(d), index=False)

Computing transition probabilities:   0%|          | 0/3767 [00:00<?, ?it/s]

Generating walks (CPU: 1):  28%|██▊       | 28/100 [01:01<02:45,  2.30s/it]

KeyboardInterrupt: 

### Weighted Features

In [97]:
# NCA for kaggle: G
NCAD = aPairs

# NCA for test: G_test
NCAD_test = NCAD
for pair in test.loc[test['Exist'] == 1].Pair:
    if pair in NCAD_test:
        NCAD_test[pair] -= 1

# NCA for train: G_train
NCAD_train = NCAD_test
for pair in train.loc[train['Exist'] == 1].Pair:
    if pair in NCAD_train:
        NCAD_train[pair] -= 1

In [103]:
# calculate WCN
def wcn(G, u, v, alpha):
    wcn = 0
    cnlist = list(sorted(nx.common_neighbors(G,u,v)))
    if len(cnlist) == 0:
        return wcn
    else:
        for cn in cnlist:
            wcn += ((G[u][cn]['weight'])**alpha) + ((G[v][cn]['weight'])**alpha)
        return wcn

In [104]:
# calculate WAA
def waa(G, u, v, alpha):
    waa = 0
    cnlist = list(sorted(nx.common_neighbors(G,u,v)))
    if len(cnlist) == 0:
        return waa
    else:
        for cn in cnlist:
            w = ((G[u][cn]['weight'])**alpha) + ((G[v][cn]['weight'])**alpha)
            cn_neighbors = list(sorted(G.neighbors(cn)))
            s = 0
            for ne in cn_neighbors:
                s += G[cn][ne]['weight']**alpha
            waa += w/(np.log(1+s))
        return waa

In [105]:
# calculate WRA
def wra(G, u, v, alpha):
    wra = 0
    cnlist = list(sorted(nx.common_neighbors(G,u,v)))
    if len(cnlist) == 0:
        return wra
    else:
        for cn in cnlist:
            w = ((G[u][cn]['weight'])**alpha) + ((G[v][cn]['weight'])**alpha)
            cn_neighbors = list(sorted(G.neighbors(cn)))
            s = 0
            for ne in cn_neighbors:
                s += G[cn][ne]['weight']**alpha
            wra += w/s
        return wra

#### Train

In [106]:
# add nca as weight to all edges
nx.set_edge_attributes(G_train, values = NCAD_train, name = 'weight')

In [63]:
alphas = [-4.0, -3.0] + [round(a*0.1,1) for a in range(-20, 21)] + [3.0, 4.0]
#df = train
alphas

[-4.0,
 -3.0,
 -2.0,
 -1.9,
 -1.8,
 -1.7,
 -1.6,
 -1.5,
 -1.4,
 -1.3,
 -1.2,
 -1.1,
 -1.0,
 -0.9,
 -0.8,
 -0.7,
 -0.6,
 -0.5,
 -0.4,
 -0.3,
 -0.2,
 -0.1,
 0.0,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 1.0,
 1.1,
 1.2,
 1.3,
 1.4,
 1.5,
 1.6,
 1.7,
 1.8,
 1.9,
 2.0,
 3.0,
 4.0]

In [110]:
# Generate weighted features for different alphas accoring to train
# and save to folder alphas as weak_ties_(alpha).csv
for alpha in alphas:
    WCN = []
    WAA = []
    WRA = []
    for u,v in tqdm(df.Pair):
        cn = wcn(G_train, u, v, alpha)
        WCN.append(((u,v), cn))
        
        aa = waa(G_train, u, v, alpha)
        WAA.append(((u,v), aa))
        
        ra = wra(G_train, u, v, alpha)
        WRA.append(((u,v), ra))
        
    wcnDF = pd.DataFrame(WCN, columns=["Pair", "WCN"])
    waaDF = pd.DataFrame(WAA, columns=["Pair", "WAA"])
    wraDF = pd.DataFrame(WRA, columns=["Pair", "WRA"])
    
    weak_ties = wcnDF.join(waaDF.set_index('Pair'), on="Pair")
    weak_ties = weak_ties.join(wraDF.set_index('Pair'), on="Pair")
    weak_ties.to_csv("alphas/weak_ties_{}.csv".format(alpha), index=False)


100%|██████████| 2886/2886 [00:01<00:00, 1969.36it/s]

100%|██████████| 2886/2886 [00:01<00:00, 1985.50it/s]

100%|██████████| 2886/2886 [00:01<00:00, 1998.06it/s]

100%|██████████| 2886/2886 [00:01<00:00, 1994.08it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2018.14it/s]

100%|██████████| 2886/2886 [00:01<00:00, 1938.53it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2038.52it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2050.43it/s]

100%|██████████| 2886/2886 [00:01<00:00, 1854.63it/s]

100%|██████████| 2886/2886 [00:01<00:00, 1881.74it/s]

100%|██████████| 2886/2886 [00:01<00:00, 1939.84it/s]

100%|██████████| 2886/2886 [00:01<00:00, 1908.88it/s]

100%|██████████| 2886/2886 [00:01<00:00, 1944.55it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2061.66it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2109.36it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2116.73it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2108.98it/s]

100%|██████████| 2886/2886 [00:01<00:00, 1978.58it/s]

100%|████

100%|██████████| 2886/2886 [00:01<00:00, 2081.81it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2120.37it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2165.80it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2151.69it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2150.34it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2146.00it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2024.17it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2039.29it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2037.65it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2004.72it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2002.83it/s]

100%|██████████| 2886/2886 [00:01<00:00, 1979.50it/s]

100%|██████████| 2886/2886 [00:01<00:00, 1990.82it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2028.39it/s]

100%|██████████| 2886/2886 [00:01<00:00, 1981.25it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2008.09it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2027.68it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2040.22it/s]

100%|█████

100%|██████████| 2886/2886 [00:01<00:00, 2104.98it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2135.21it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2139.75it/s]

100%|██████████| 2886/2886 [00:01<00:00, 2096.11it/s]


#### Test

In [107]:
# add nca as weight to all edges
nx.set_edge_attributes(G_test, values = NCAD_test, name = 'weight')

In [ ]:
alpha = 0.1 # fill in your own number
df = test

WCN = []
WAA = []
WRA = []
for u,v in tqdm(df.Pair):
    cn = wcn(G_test, u, v, alpha)
    WCN.append(((u,v), cn))
        
    aa = waa(G_test, u, v, alpha)
    WAA.append(((u,v), aa))
        
    ra = wra(G_test, u, v, alpha)
    WRA.append(((u,v), ra))
        
wcnDF = pd.DataFrame(WCN, columns=["Pair", "WCN"])
waaDF = pd.DataFrame(WAA, columns=["Pair", "WAA"])
wraDF = pd.DataFrame(WRA, columns=["Pair", "WRA"])
    
test_w = wcnDF.join(waaDF.set_index('Pair'), on="Pair")
test_w = test_w.join(wraDF.set_index('Pair'), on="Pair")
test_w.to_csv("test_w_{}.csv".format(alpha), index=False)

#### Kaggle

In [108]:
# add nca as weight to all edges
nx.set_edge_attributes(G, values = NCAD, name = 'weight')

In [ ]:
alpha = 0.1 # fill in your own number
df = kaggle

WCN = []
WAA = []
WRA = []
for u,v in tqdm(df.Pair):
    cn = wcn(G, u, v, alpha)
    WCN.append(((u,v), cn))
        
    aa = waa(G, u, v, alpha)
    WAA.append(((u,v), aa))
        
    ra = wra(G, u, v, alpha)
    WRA.append(((u,v), ra))
        
wcnDF = pd.DataFrame(WCN, columns=["Pair", "WCN"])
waaDF = pd.DataFrame(WAA, columns=["Pair", "WAA"])
wraDF = pd.DataFrame(WRA, columns=["Pair", "WRA"])
    
kaggle_w = wcnDF.join(waaDF.set_index('Pair'), on="Pair")
kaggle_w = kaggle_w.join(wraDF.set_index('Pair'), on="Pair")
kaggle_w.to_csv("kaggle_w_{}.csv".format(alpha), index=False)

## Data Exploration

In [ ]:
# correlation plot
# corr_X = X.corr()
# plt.figure(figsize=(8, 6))
# heatmap(corr_X, cmap="YlGnBu")

# boxplot of NCA
# boxplot(x="Exist", y="NCA", data=train[train["NCA"] < 8])

# boxplot of CN
# boxplot(x="Exist", y="CN", data=train)

# boxplot of AA
# boxplot(x="Exist", y="AA", data=train)

# boxplot of RA
# boxplot(x="Exist", y="RA", data=train)

# boxplot of JC
# boxplot(x="Exist", y="JC", data=train)

# boxplot of PA
# boxplot(x="Exist", y="PA", data=train)

# boxplot of KI
# boxplot(x="Exist", y="KI", data=train[train['KI']>-1][train['KI']<1])